# model L4

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L4, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

train_mask = df_feat_raw.index.to_numpy() < date_to_timestamp("2025-03-01")
train_raw = df_feat_raw.iloc[train_mask]
train_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])
train_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])

df_feat_l4 = pd.concat([train_deep_ssm, train_lg_ssm, train_raw], axis=1)[FEAT_L4]
print(f"{df_feat_l4.shape = }")

label = np.load("data/label_hard_L4.npy")
train_y = label[train_mask]

assert df_feat_l4.shape[0] == train_y.shape[0]

print(np.unique(train_y, return_counts=True))

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json
df_feat_l4.shape = (4632, 2179)
(array([0, 1]), array([2079, 2553]))


In [2]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

METRIC = "f1"

def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


def objective(trial):
    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(df_feat_l4, train_y)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-09-17 21:04:17,037] A new study created in memory with name: no-name-068ba83f-2a0f-489b-8277-bb61e7f65904
[I 2025-09-17 21:04:23,360] Trial 0 finished with value: 0.8196253244397497 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 149, 'max_depth': 48, 'min_gain_to_split': 0.9360050513466003, 'min_data_in_leaf': 55, 'lambda_l1': 11.507402995321332, 'lambda_l2': 83.64859550120651, 'num_boost_round': 601}. Best is trial 0 with value: 0.8196253244397497.
[I 2025-09-17 21:04:30,073] Trial 1 finished with value: 0.799685104692996 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 279, 'max_depth': 886, 'min_gain_to_split': 0.9688129367227278, 'min_data_in_leaf': 497, 'lambda_l1': 70.1241286318485, 'lambda_l2': 8.445354526377793, 'num_boost_round': 777}. Best is trial 0 with value: 0.8196253244397497.
[I 2025-09-17 21:05:07,762] Trial 2 finished with value: 0.8163893544231788 and parameters: {'i

In [4]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 264,
 'max_depth': 193,
 'min_gain_to_split': 0.0772267735811633,
 'min_data_in_leaf': 460,
 'lambda_l1': 0.01394889855932191,
 'lambda_l2': 1.2708306044559705,
 'num_boost_round': 1213}

In [5]:
params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(df_feat_l4, train_y)
side_model = lgb.train(params, dtrain)
# side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [6]:
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L4, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_l4_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L4]

label_full = np.load("data/label_hard_L4.npy")

dtrain = lgb.Dataset(df_feat_l4_full, label_full)
side_model_prod = lgb.train(params, dtrain)

Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [7]:
import plotly.express as px

test_features = df_feat_l4_full[df_feat_l4_full.index.to_numpy() >= date_to_timestamp("2025-03-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
side_model.save_model("strategies/BinanceBtcDeapV1Voting/models/model_l4.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcDeapV1Voting/models/model_l4_prod.txt"
)

# model L5

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L5, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L5.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

train_mask = df_feat_raw.index.to_numpy() < date_to_timestamp("2025-03-01")
train_raw = df_feat_raw.iloc[train_mask]
train_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])
train_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])

df_feat_l5 = pd.concat([train_deep_ssm, train_lg_ssm, train_raw], axis=1)[FEAT_L5]
print(f"{df_feat_l5.shape = }")

label = np.load("data/label_hard_L5.npy")
train_y = label[train_mask]

assert df_feat_l5.shape[0] == train_y.shape[0]

print(np.unique(train_y, return_counts=True))

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json
df_feat_l5.shape = (4632, 2507)
(array([0, 1]), array([2015, 2617]))


In [2]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

METRIC = "f1"

def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


def objective(trial):
    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(df_feat_l5, train_y)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-09-17 22:08:03,412] A new study created in memory with name: no-name-2921757b-9454-4734-b6e6-f68d3385e180
[I 2025-09-17 22:08:08,280] Trial 0 finished with value: 0.752020220012888 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 181, 'max_depth': 741, 'min_gain_to_split': 0.31219390867324776, 'min_data_in_leaf': 488, 'lambda_l1': 84.19426814780631, 'lambda_l2': 9.058874993254651, 'num_boost_round': 472}. Best is trial 0 with value: 0.752020220012888.
[I 2025-09-17 22:08:26,067] Trial 1 finished with value: 0.7542702608590519 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 186, 'max_depth': 615, 'min_gain_to_split': 0.9451672659006287, 'min_data_in_leaf': 380, 'lambda_l1': 73.46727629669921, 'lambda_l2': 58.6555490002225, 'num_boost_round': 831}. Best is trial 1 with value: 0.7542702608590519.
[I 2025-09-17 22:08:29,253] Trial 2 finished with value: 0.7573148784412178 and parameters: {

In [4]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': False,
 'extra_trees': True,
 'boosting': 'gbdt',
 'num_leaves': 207,
 'max_depth': 993,
 'min_gain_to_split': 0.004161385684743585,
 'min_data_in_leaf': 90,
 'lambda_l1': 0.16935187787521117,
 'lambda_l2': 0.18723971207387535,
 'num_boost_round': 1085}

In [5]:
params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(df_feat_l5, train_y)
side_model = lgb.train(params, dtrain)

In [6]:
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L5, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L5.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_l5_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L5]

label_full = np.load("data/label_hard_L5.npy")

dtrain = lgb.Dataset(df_feat_l5_full, label_full)
side_model_prod = lgb.train(params, dtrain)

Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [7]:
import plotly.express as px

test_features = df_feat_l5_full[df_feat_l5_full.index.to_numpy() >= date_to_timestamp("2025-03-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
side_model.save_model("strategies/BinanceBtcDeapV1Voting/models/model_l5.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcDeapV1Voting/models/model_l5_prod.txt"
)

# model L6

In [2]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L6, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L6.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

train_mask = df_feat_raw.index.to_numpy() < date_to_timestamp("2025-03-01")
train_raw = df_feat_raw.iloc[train_mask]
train_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])
train_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF].iloc[train_mask])

df_feat_l6 = pd.concat([train_deep_ssm, train_lg_ssm, train_raw], axis=1)[FEAT_L6]
print(f"{df_feat_l6.shape = }")

label = np.load("data/label_hard_L6.npy")
train_y = label[train_mask]

assert df_feat_l6.shape[0] == train_y.shape[0]

print(np.unique(train_y, return_counts=True))

Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json
df_feat_l6.shape = (4632, 458)
(array([0, 1]), array([2039, 2593]))


In [3]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score

METRIC = "f1"

def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


def objective(trial):
    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(df_feat_l6, train_y)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-09-17 23:36:29,774] A new study created in memory with name: no-name-e876ebf6-307f-4db5-b063-00cd9d005427
[I 2025-09-17 23:36:33,508] Trial 0 finished with value: 0.7079389623084031 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 67, 'max_depth': 86, 'min_gain_to_split': 0.9259014243117817, 'min_data_in_leaf': 140, 'lambda_l1': 58.33284087650563, 'lambda_l2': 67.98093341610829, 'num_boost_round': 748}. Best is trial 0 with value: 0.7079389623084031.
[I 2025-09-17 23:37:04,388] Trial 1 finished with value: 0.7103218574508822 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 137, 'max_depth': 519, 'min_gain_to_split': 0.7733915868064228, 'min_data_in_leaf': 23, 'lambda_l1': 95.85534469264913, 'lambda_l2': 90.9643392016736, 'num_boost_round': 1435}. Best is trial 1 with value: 0.7103218574508822.
[I 2025-09-17 23:37:09,494] Trial 2 finished with value: 0.7097613082258281 and parameters: {'i

In [5]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 102,
 'max_depth': 942,
 'min_gain_to_split': 0.016293100493589244,
 'min_data_in_leaf': 87,
 'lambda_l1': 1.626035317109549,
 'lambda_l2': 45.58793271832796,
 'num_boost_round': 398}

In [6]:
params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(df_feat_l6, train_y)
side_model = lgb.train(params, dtrain)

In [7]:
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L6, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L6.parquet")
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_l6_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L6]

label_full = np.load("data/label_hard_L6.npy")

dtrain = lgb.Dataset(df_feat_l6_full, label_full)
side_model_prod = lgb.train(params, dtrain)

Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [8]:
import plotly.express as px

test_features = df_feat_l6_full[df_feat_l6_full.index.to_numpy() >= date_to_timestamp("2025-03-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
side_model.save_model("strategies/BinanceBtcDeapV1Voting/models/model_l6.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcDeapV1Voting/models/model_l6_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(f"{np.unique(meta_label, return_counts=True) = }")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)
print(f"{np.unique(meta_label_masked, return_counts=True) = }")

meta_features.isna().sum(axis=0).sort_values(ascending=False)

np.unique(meta_label, return_counts=True) = (array([0., 1.]), array([  669, 12550]))
(13219, 2664)
(13219,)
(12115, 2664)
(12115,)
np.unique(meta_label_masked, return_counts=True) = (array([0., 1.]), array([  632, 11483]))


approximate_entropy_win128_spot    0
cwt_win1024_13_lag2                0
cwt_win256_11_dt_lag8              0
cwt_win128_8_lag4                  0
cwt_win64_12_dt                    0
                                  ..
cwt_win512_20_dt_lag11             0
cwt_win1024_11_dt_lag14            0
cwt_win128_1_dt_lag2               0
adx_7_lag3                         0
model                              0
Length: 2664, dtype: int64

In [2]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,
    fbeta_score,
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="weighted")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [3]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1500),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(n_startup_trials=50),
)
study.optimize(objective, n_trials=200, n_jobs=1)

[I 2025-07-01 10:10:04,971] A new study created in memory with name: no-name-1188c647-119a-443b-b088-01c9353277eb
[I 2025-07-01 10:10:12,087] Trial 0 finished with value: 0.924743520258233 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 140, 'max_depth': 951, 'min_gain_to_split': 0.8206526039318547, 'min_data_in_leaf': 201, 'lambda_l1': 12.089838569582271, 'lambda_l2': 95.92885685399514, 'num_boost_round': 193}. Best is trial 0 with value: 0.924743520258233.
[I 2025-07-01 10:10:32,056] Trial 1 finished with value: 0.924743520258233 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 72, 'max_depth': 576, 'min_gain_to_split': 0.5274289677732332, 'min_data_in_leaf': 250, 'lambda_l1': 36.60501958746357, 'lambda_l2': 9.341901524626236, 'num_boost_round': 1043}. Best is trial 0 with value: 0.924743520258233.
[I 2025-07-01 10:10:37,441] Trial 2 finished with value: 0.9280243308865458 and parameters: {'i

In [4]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 234,
 'max_depth': 579,
 'min_gain_to_split': 0.4783776963600167,
 'min_data_in_leaf': 254,
 'lambda_l1': 5.143714268925239,
 'lambda_l2': 21.356313125652914,
 'num_boost_round': 715}

In [5]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [6]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [7]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")